In [4]:
import librosa.display
import librosa.feature
import numpy as np
import scipy.signal


# t-2 Voice Activity Detection function
def detect_activity(y, sr,
        n_mels=128, fmin=1000, fmax=11025, 
        hop_length=512, gain=0.8, bias=10, power=0.25, pcen_time_constant=0.06, eps=1e-06,
        medfilt_time_constant=None, normalized=True,
        peak_threshold=0.45, activity_threshold=0.2):
    
    # 1. Compute mel-frequency spectrogram
    melspec = librosa.feature.melspectrogram(
        y, sr=sr, fmin=fmin, fmax=fmax, hop_length=hop_length,
        n_mels=n_mels)
    
    # 2. Compute per-channel energy normalization (PCEN-SNR)
    pcen = librosa.core.pcen(melspec, sr=sr, gain=gain, bias=bias,
        power=power, hop_length=hop_length,
        time_constant=pcen_time_constant, eps=eps)
    
    # 3. compute PCEN-SNR detection function
    pcen_snr = np.max(pcen,axis=0) - np.min(pcen,axis=0)
    pcen_snr = librosa.power_to_db(pcen_snr / np.median(pcen_snr))
    if normalized:
        pcen_snr = pcen_snr / np.max(pcen_snr)
        
    # 4. Apply median filtering.
    if medfilt_time_constant is not None:
        medfilt_hops = medfilt_time_constant * sr / hop_length
        kernel_size = max(1, 1 + 2 * round(medfilt_hops - 0.5))
        pcen_snr = scipy.signal.medfilt(pcen_snr, kernel_size=kernel_size)
    
    # 5. Extract active segments.
    activity, start, end = threshold_activity(
        pcen_snr, peak_threshold, activity_threshold)
    
    # 6. Convert indices to seconds.
    start_times = np.round(np.array(start) * hop_length / sr, 3)
    end_times = np.round(np.array(end) * hop_length / sr, 3)
    
    return start_times, end_times, pcen


def threshold_activity(x, Tp, Ta):
    locs = scipy.signal.find_peaks(x,height = Tp)[0]
    y = (x > Ta) * 1
    act = np.diff(y)
    u = np.where(act == 1)[0]
    d = np.where(act == -1)[0]
    signal_length = len(x)
    
    if d[0] < u[0]:
        u = np.insert(u, 0, 0)
        
    if d[-1] < u[-1]:
        d = np.append(d, signal_length-1)
        
    starts = []
    ends = []
    
    activity = np.zeros(signal_length,)
    
    for candidate_up, candidate_down in zip(u, d):
        candidate_segment = range(candidate_up, candidate_down)
        peaks_in_segment = [x in candidate_segment for x in locs]
        is_valid_candidate = np.any(peaks_in_segment)
        if is_valid_candidate:
            starts.append(candidate_up)
            ends.append(candidate_down)
            activity[candidate_segment] = 1.0
            
    starts = np.array(starts)
    ends = np.array(ends)
    
    return activity, starts, ends


def get_events_time(rdata):

    rdata_nz = np.nonzero(rdata)[0][-1]
    fdata = rdata[0:rdata_nz]
        
    S = librosa.feature.melspectrogram(fdata,n_mels = 256,n_fft=2048, hop_length = 512)
    pcen = librosa.core.pcen(S, sr=44100,
    gain=0.8, bias=10, power=0.25, time_constant=0.06, eps=1e-06)
    
    pcen_snr = np.max(pcen,axis=0) - np.min(pcen,axis=0)
    pcen_snr = librosa.power_to_db(pcen_snr / np.median(pcen_snr))
    pcen_snr = pcen_snr / np.max(pcen_snr)
    median_pcen_snr = scipy.signal.medfilt(pcen_snr, kernel_size=3)
    
    activity, start, end = threshold_activity(median_pcen_snr,0.45,0.15)
    activity_count = len(start)
    
    return start, end, pcen


def get_coef(start, end, pcen):

    a_all = []
    b_all = []
    max_list = []
    min_list = []
    poly_coef = []
        
    for s,e in zip(start,end):   
        bins = np.argmax(pcen[:,s:e],0)
        bins_filt = bins[(bins < np.mean(bins) + 2*np.std(bins)) & (bins > np.mean(bins) - 2*np.std(bins)) & (bins<200)& (bins>5)]

        if len(bins_filt) != 0:
            x = np.array(range(0,len(bins_filt)))
            x = x - np.mean(x)
            z = np.polyfit(x, bins_filt, 2)

            p = np.poly1d(z)
            a = z[0]
            b = z[1]
            poly_coef = np.append(poly_coef, z[0:2])
    
            # Store needed variables - coefficients, max and min
            a_all = np.append(a_all,a)
            b_all = np.append(b_all,b)
            xp = np.linspace(x.min(), x.max(), len(bins_filt))
            max_list = np.append(max_list,x.max())
            min_list = np.append(min_list,x.min())

    return a_all, b_all, max_list, min_list




def pitch_contour(a_sti_all,b_sti_all,max_sti_list,min_sti_list):
    sti_states = []
    for a,b,amax,amin in zip(a_sti_all,b_sti_all,max_sti_list,min_sti_list):
        if a > 0.01:
            if -b / (2*a) <= amin:
                seq = 'uuu'
            elif 0 >= -b / (2*a) > amin :
                seq = 'duu'
            elif amax > -b / (2*a) > 0:
                seq = 'ddu'
            else:
                seq = 'ddd'
            
        elif a < -0.01:
            if -b / (2*a) <= amin:
                seq = 'ddd'
            elif 0 >= -b / (2*a) > amin :
                seq = 'udd'
            elif amax > -b / (2*a) > 0:
                seq = 'uud'
            else:
                seq = 'uuu'
            
        else:
            if b > 0.1:
                seq = 'uuu'
            elif b < -0.1:
                seq = 'ddd'
            else:
                seq = 'fff'
                
        sti_states.append(seq)  
    return sti_states